#  Team members
**1) CS21M024 -Joyojyoti Acharya** 

**2) CS21M075-  Vrushab Karia**


#  Q8) Generating english lyrics by Fine tuning GPT-2


Reference:- https://towardsdatascience.com/natural-language-generation-part-2-gpt-2-and-huggingface-f3acb35bc86a

We have refered this blog for fine tuning GPT-2.

**We are cloning the transformers github repository**

In [3]:
#Cloning the transformers github repository
!git clone https://github.com/huggingface/transformers

**Checking the GPU allocated to us**

In [4]:
!nvidia-smi

**We are Changing the directory location to be in the examples folder and then install any requirements**

In [5]:
import os
os.chdir("transformers")
os.chdir("./examples/tensorflow/language-modeling")
!pip install -r requirements.txt
!pip install pyarrow --upgrade
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+"https://github.com/huggingface/transformers/"

In [6]:
#Use this if your using the google collab notebook
# from google.colab import drive 
# drive.mount('/content/gdrive')

**Preprocessing the input data**

**We runned the below script in the local machine to concatenate all the lyrics file into one merged file. We have used the dataset 2 given in the question and merged all the files in that dataset.**

In [7]:
# import os
# # find all the txt files in the dataset folder
# inputs = []
# for file in os.listdir("archive"):
#     if file.endswith(".txt"):
#         inputs.append(os.path.join("archive", file))
 
 
# # concatanate all txt files in a file called merged_file.txt
# with open('merged_file.txt', 'w',encoding="utf-8") as outfile:
#     for fname in inputs:
#         with open(fname, encoding="utf-8", errors='ignore') as infile:
#             outfile.write(infile.read())
# print("done")

In [8]:
"""
loading the data line by line
"""



from sklearn.model_selection import train_test_split

#Please add the location of the merged_file.txt dataset here
with open('/kaggle/input/merged-file/merged_file.txt', 'r') as data:
  dataset = ["" + x.strip() for x in data.readlines()]

train, evalu = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(evalu)))

with open('/kaggle/working/train_tmp.txt', 'w') as file_handle:
  file_handle.write(" ".join(train))

with open('/kaggle/working/eval_tmp.txt', 'w') as file_handle:
  file_handle.write(" ".join(evalu))

# **Fine tunning the GPT-2 Model**

**Note :- We have used 'distil GPT-2' which is a smaller version of the 'GPT-2' model instead of normal GPT2. This is because the original GPT-2 model is very huge and we could not load and train it given the computing resource we had. So due to the resource constraints we have used distil GPT-2. However it is advisable to use original GPT-2 if you have no resource constraints**

In [9]:
!python run_clm.py \
--model_type distilgpt2 \
--model_name_or_path distilgpt2\
--train_file "/kaggle/working/train_tmp.txt" \
--do_train \
--validation_file "/kaggle/working/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/kaggle/working/question_8_op"

# **Now lets use the fine tunned model**.

In [10]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/kaggle/working/question_8_op", from_pt=False)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")


# Passing and Encoding the sample text 

In [11]:
s="I love deep learning," # This is the initial prompt to the model
input_ids = tokenizer.encode(s, return_tensors='tf')

#  **Generating the lyrics from our input smaple**. 
Reference used for parameter training https://huggingface.co/blog/how-to-generate

Note:- You can play around with the paramters to get some better outputs.

In [12]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=200,  
    num_return_sequences=5,
    #no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

# **Printing the lyrics**

In [13]:
#Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

# **Thankyou**